In [1]:
from decord import VideoReader, cpu
import numpy as np

from transformers import VideoMAEFeatureExtractor, VideoMAEModel
from huggingface_hub import hf_hub_download

/media/wtheisen/scratch3/triclip/triple_clip_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len

    selected_image_frame = start_idx
    while selected_image_frame >= start_idx and selected_image_frame <= end_idx:
        selected_image_frame = np.random.randint(0, seg_len)

    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices, selected_image_frame

In [20]:
video_path = "/media/wtheisen/scratch3/social-data/./media_2oct/es/1444262872394182657/videos/link/1444262872394182657.mp4"
videoreader = VideoReader(video_path, num_threads=1, ctx=cpu(0))


In [23]:
# sample 16 frames
videoreader.seek(0)
indices, image_frame = sample_frame_indices(clip_len=16, frame_sample_rate=4, seg_len=len(videoreader))
video = videoreader.get_batch(indices).asnumpy()
print(videoreader[image_frame].asnumpy())

64
[ 76  80  84  88  93  97 101 105 110 114 118 122 127 131 135 139]
[[[ 18  17  22]
  [ 18  17  22]
  [ 18  17  22]
  ...
  [ 26  19  17]
  [ 26  19  17]
  [ 26  19  17]]

 [[ 18  17  22]
  [ 18  17  22]
  [ 18  17  22]
  ...
  [ 26  19  17]
  [ 26  19  17]
  [ 24  17  15]]

 [[ 18  17  22]
  [ 18  17  22]
  [ 18  17  22]
  ...
  [ 26  19  17]
  [ 24  17  15]
  [ 24  17  15]]

 ...

 [[133 102  83]
  [133 102  83]
  [133 102  83]
  ...
  [ 62  49  42]
  [ 61  48  43]
  [ 61  48  43]]

 [[133 102  83]
  [133 102  83]
  [133 102  83]
  ...
  [ 61  48  41]
  [ 61  48  43]
  [ 61  48  43]]

 [[133 102  83]
  [133 102  83]
  [133 102  83]
  ...
  [ 61  48  41]
  [ 61  48  43]
  [ 61  48  43]]]


AttributeError: 'NoneType' object has no attribute 'asnumpy'

In [13]:
feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

In [14]:
# prepare video for the model
inputs = feature_extractor(list(video), return_tensors="pt")

In [15]:
# forward pass
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

[1, 1568, 768]